In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import deque
from surprise import Dataset, NormalPredictor, Reader
from surprise.model_selection import cross_validate
import os
from sklearn.model_selection import train_test_split
import pickle
from surprise import accuracy
from surprise import accuracy, Dataset, Reader, SVD, KNNBaseline
from surprise.model_selection import PredefinedKFold
import pprint
import tempfile
from typing import Dict, Text
import tensorflow as tf
import tensorflow_recommenders as tfrs
from collections import defaultdict
import joblib

In [2]:
#!pwd

/Users/alecclarkfeather/capstone_project


In [3]:
PATH = '/Users/alecclarkfeather/capstone_project/'

train_hr0 = pd.read_csv(PATH + 'hit_rate_folds_actual/train_hr0.csv', header=None)
test_hr0 = pd.read_csv(PATH + 'hit_rate_folds_actual/test_hr0.csv', header=None)

In [4]:
train_hr0.columns = ['user_id', 'movie_id', 'rating']
test_hr0.columns = ['user_id', 'movie_id', 'rating']

In [5]:
train_hr0.head()

,user_id,movie_id,rating
0,826574,9729,4.0
1,2200645,1719,4.0
2,2150434,5356,5.0
3,2142065,8753,3.0
4,867086,6721,4.0


In [6]:
test_hr0.head()

,user_id,movie_id,rating
0,1667994,7904,3.0
1,1105606,10359,2.0
2,2129781,8636,4.0
3,1129575,2519,4.0
4,1001105,8966,4.0


In [7]:
top_ratings_df = pd.read_csv(PATH + 'top_ratings_df.csv')
top_ratings_df.head()

,user_id,movie_id,rating
0,841872,14454,5.0
1,689085,299,5.0
2,2529854,11165,5.0
3,2597445,10886,4.0
4,2336656,3282,5.0


In [8]:
len(top_ratings_df) == len(top_ratings_df['user_id'].unique())

True

In [10]:
hr_model_0a = joblib.load(open(PATH + 'model0a.pkl', 'rb'))

In [11]:
#type(hr_model_0a)

keras.saving.saved_model.load.NetflixRmseModel

In [12]:
#hr_model_0a.summary()

Model: "netflix_rmse_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 deep_rec_rmse_model_4 (Deep  multiple                 4449105   
 RecRmseModel)                                                   
                                                                 
 ranking_4 (Ranking)         multiple                  0         
                                                                 
Total params: 4,449,105
Trainable params: 4,449,105
Non-trainable params: 0
_________________________________________________________________


In [13]:
top_ratings_df['user_id'].loc[0]

841872

In [14]:
test_pred = hr_model_0a({'user_id': np.array([top_ratings_df['user_id'].loc[0]]),
                        'movie_id': np.array([top_ratings_df['movie_id'].loc[0]])})

In [30]:
test_pred.numpy()[0][0]

3.6029956

In [39]:
top_ratings_df['user_id'].values.shape

(264059,)

In [40]:
np.random.randn(264059).shape

(264059,)

In [41]:
np.sqrt(np.mean((top_ratings_df['user_id'].values - np.random.randn(264059))**2))

1529188.2514990915

In [42]:
xyz = [1,2,3,4]
np.array(xyz).shape

(4,)

In [38]:
#predictions_1a = defaultdict(list)

#for i in range(len(test_hr0)):
    #user = test_hr0['user_id'].loc[i]
    #movie = test_hr0['movie_id'].loc[i]
    #predicted_rating = hr_model_0a({'user_id': np.array([user]),
                                           #'movie_id': np.array([movie])})
   # predictions_1a[user].append((movie, predicted_rating.numpy()[0][0]))

In [37]:
#len(predictions_1a)
#predictions_1a[1667994]

[(7904, 2.782896),
 (16703, 3.440704),
 (12435, 3.8466794),
 (3148, 3.601721),
 (3446, 4.0607963),
 (8159, 3.6908143),
 (3320, 3.529143),
 (16108, 2.7437606),
 (16223, 4.0090284),
 (1220, 3.9485912),
 (5805, 3.9380841),
 (963, 3.8927152),
 (9108, 2.6777651),
 (16969, 3.9435968)]

In [ ]:
hr_model_0b = joblib.load(open(PATH + 'model0b.pkl', 'rb'))

predictions_1b = defaultdict(list)

for i in range(len(train_hr0)):
    user = train_hr0['user_id'].loc[i]
    movie = train_hr0['movie_id'].loc[i]
    predicted_rating = hr_model_0b({'user_id': np.array([user]),
                                           'movie_id': np.array([movie])})
    predictions_1b[user].append((movie, predicted_rating.numpy()[0][0]))

In [ ]:
for user_id, user_ratings in predictions_1a.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    predictions_1a[user_id] = user_ratings[:100]

In [ ]:
for user_id, user_ratings in predictions_1b.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    predictions_1b[user_id] = user_ratings[:100]

In [ ]:
num_users = 0
hits = 0

for user_id in predictions_1a.keys():
    top_movie = top_ratings_df[top_ratings_df['user_id']==user_id]['movie_id']
    num_users += 1
    num_recs = len(predictions_1a[user_id])
    top_number = int(num_recs*0.25)
    boundary_for_user = [rec[1] for rec in predictions_1a[user_id]][top_number]
    pred_top_movie = hr_model_0a({'user_id': np.array([user_id]),
                                         'movie_id': np.array([top_movie])})
    if pred_top_movie.numpy()[0][0] > boundary_for_user:
        hits += 1
        
hit_rate_1a = hits / num_users
print(hit_rate_1a)

In [ ]:
num_users = 0
hits = 0

for user_id in predictions_1b.keys():
    top_movie = top_ratings_df[top_ratings_df['user_id']==user_id]['movie_id']
    num_users += 1
    num_recs = len(predictions_1b[user_id])
    top_number = int(num_recs*0.25)
    boundary_for_user = [rec[1] for rec in predictions_1b[user_id]][top_number]
    pred_top_movie = hr_model_0b({'user_id': np.array([user_id]),
                                         'movie_id': np.array([top_movie])})
    if pred_top_movie.numpy()[0][0] > boundary_for_user:
        hits += 1
        
hit_rate_1b = hits / num_users
print(hit_rate_1b)

In [ ]:
hit_rates_nn = []
hit_rates_nn.append(hit_rate_1a)
hit_rates_nn.append(hit_rate_1b)

for i in range(1, 5):
    hr_model_1a = joblib.load(open(PATH + f'model{i}a.pkl', 'rb'))
    hr_model_1b = joblib.load(open(PATH + f'model{i}b.pkl', 'rb'))
    train_hr = pd.read_csv(PATH + f'hit_rate_folds_actual/train_hr{i}.csv', header=None)
    test_hr = pd.read_csv(PATH + f'hit_rate_folds_actual/test_hr{i}.csv', header=None)
    train_hr.columns = ['user_id', 'movie_id', 'rating']
    test_hr.columns = ['user_id', 'movie_id', 'rating']
    
    predictions_a = defaultdict(list)
    for i in range(len(test_hr)):
        user = test_hr['user_id'].loc[i]
        movie = test_hr['movie_id'].loc[i]
        predicted_rating = hr_model_1a({'user_id': np.array([user]),
                                           'movie_id': np.array([movie])})
        predictions_a[user].append((movie, predicted_rating.numpy()[0][0]))
        
    for user_id, user_ratings in predictions_a.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        predictions_a[user_id] = user_ratings[:100]
        
    predictions_b = defaultdict(list)

    for i in range(len(train_hr)):
        user = train_hr['user_id'].loc[i]
        movie = train_hr['movie_id'].loc[i]
        predicted_rating = hr_model_1b({'user_id': np.array([user]),
                                           'movie_id': np.array([movie])})
        predictions_b[user].append((movie, predicted_rating.numpy()[0][0]))
        
    
    for user_id, user_ratings in predictions_b.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        predictions_b[user_id] = user_ratings[:100]
        
    num_users_a = 0
    num_users_b = 0
    num_hits_a = 0
    num_hits_b = 0
    
    for user_id in predictions_a.keys():
        top_movie = top_ratings_df[top_ratings_df['user_id']==user_id]['movie_id']
        num_users_a += 1
        num_recs = len(predictions_a[user_id])
        top_number = int(num_recs*0.25)
        boundary_for_user = [rec[1] for rec in predictions_a[user_id]][top_number]
        pred_top_movie = hr_model_1a({'user_id': np.array([user_id]),
                                         'movie_id': np.array([top_movie])})
        if pred_top_movie.numpy()[0][0] > boundary_for_user:
            num_hits_a += 1
            
    for user_id in predictions_b.keys():
        top_movie = top_ratings_df[top_ratings_df['user_id']==user_id]['movie_id']
        num_users_b += 1
        num_recs = len(predictions_b[user_id])
        top_number = int(num_recs*0.25)
        boundary_for_user = [rec[1] for rec in predictions_b[user_id]][top_number]
        pred_top_movie = hr_model_1b({'user_id': np.array([user_id]),
                                         'movie_id': np.array([top_movie])})
        if pred_top_movie.numpy()[0][0] > boundary_for_user:
            num_hits_b += 1
            
    hit_rates_nn.append(num_hits_a / num_users_a)
    hit_rates_nn.append(num_hits_b / num_users_b)
    
    
    